# PAN Number Validation Project

## Data Cleaning and Validation

### Objective 

You are tasked with cleaning and validating a dataset containing the Permanent
Account Numbers (PAN) of Indian nationals. The goal is to ensure that each PAN
number adheres to the official format and is categorised as either Valid or Invalid.
The dataset is given in a separate Excel file.

### Instructions

1. **Data Cleaning and Preprocessing:**
-   Identify and handle missing data: PAN numbers may have missing values.These missing values need to be handled appropriately, either byremoving rows or imputing values (depending on the context).

-   Check for duplicates: Ensure there are no duplicate PAN numbers. If duplicates exist, remove them.

-   Handle leading/trailing spaces: PAN numbers may have extra spaces before or after the actual number. Remove any such spaces.

-   Correct letter case: Ensure that the PAN numbers are in uppercase letters(if any lowercase letters are present).

2. **PAN Format Validation:** A valid PAN number follows the format:
  - It is exactly 10 characters long.
  - The format is as follows: AAAAA1234A
  - The first five characters should be alphabetic (uppercase letters).
      - Adjacent characters(alphabets) cannot be the same (like AABCD isinvalid; AXBCD is valid)
      - All five characters cannot form a sequence (like: ABCDE, BCDEF is invalid; ABCDX is valid)

   - The next four characters should be numeric (digits).
        - Adjacent characters(digits) cannot be the same (like 1123 is invalid; 1923 is valid)
        - All four characters cannot form a sequence (like: 1234, 2345)
        - The last character should be alphabetic (uppercase letter). Example of a valid PAN: AHGVE1276F

3. **Categorisation:**
  - Valid PAN: If the PAN number matches the above format.
  - Invalid PAN: If the PAN number does not match the correct format, is
  - incomplete, or contains any non-alphanumeric characters.

4. **Tasks:**
  - Validate the PAN numbers based on the format mentioned above.
  - **Create two separate categories:**
    - Valid PAN
    - Invalid PAN
  - **Create a summary report that provides the following:**
    - Total records processed
    - Total valid PANs
    - Total invalid PANs
    - Total missing or incomplete PANs (if applicable)


In [0]:
--  Displaying the data 

Select * from workspace.project.pan_number 

In [0]:
-- Identify and handle missing data

SELECT *
FROM workspace.project.pan_number 
WHERE Pan_Numbers IS NULL
   OR TRIM(Pan_Numbers) = '';


In [0]:
SELECT Pan_Numbers, count(*) FROM workspace.project.pan_number
GROUP by Pan_Numbers
HAVING count(*) > 1

In [0]:
-- handling leading and trailing spaces 

SELECT * from workspace.project.pan_number
where Pan_Numbers <> trim(Pan_Numbers)


In [0]:
-- Correct letter case: 

SELECT * FROM workspace.project.pan_number
WHERE Pan_Numbers <> upper(Pan_Numbers)


In [0]:
-- Cleaned Pan numbers 

SELECT DISTINCT upper(trim(Pan_Numbers)) as Pan_Numbers FROM workspace.project.pan_number 
where Pan_Numbers is not null 
and trim(Pan_Numbers) <> ''

In [0]:
-- function for repetition check 

CREATE OR REPLACE FUNCTION fn_check_adjacent_repetition(p_str STRING)
RETURNS BOOLEAN
RETURN regexp_like(p_str, '(.)\\1');


In [0]:
SELECT fn_check_adjacent_repetition('ZWOVO')

In [0]:
CREATE OR REPLACE FUNCTION fn_check_sequence(p_str STRING)
RETURNS BOOLEAN
RETURN
  CASE
    WHEN p_str IS NULL THEN FALSE
    WHEN length(p_str) < 2 THEN TRUE
    ELSE
      forall(
        sequence(1, length(p_str) - 1),
        i -> ascii(substring(p_str, i + 1, 1)) - ascii(substring(p_str, i, 1)) = 1
      )
  END;

In [0]:
SELECT fn_check_sequence('ABCDE');   -- true
-- SELECT fn_check_sequence('LMNOP');   -- true
-- SELECT fn_check_sequence('ACDFG');   -- false


In [0]:
-- valid and invalid pan categorization
Create or replace view workspace.project.vw_valid_pan 
as 
with cleaned_pan as (
SELECT DISTINCT upper(trim(Pan_Numbers)) as Pan_Numbers FROM workspace.project.pan_number 
where Pan_Numbers is not null 
and trim(Pan_Numbers) <> ''), 
Valid_pan as (
select * from cleaned_pan
where fn_check_adjacent_repetition(Pan_Numbers) = false
and 
fn_check_sequence(Substring(Pan_Numbers,1,5)) = false
and 
fn_check_sequence(Substring(Pan_Numbers,6,4)) = false
and Pan_Numbers REGEXP '^[A-Z]{5}[0-9]{4}[A-Z]$')

select cln.Pan_Numbers,
case when v_pan.Pan_Numbers is null 
then 'Invalid' 
else 'Valid' 
end as Pan_Status
from cleaned_pan cln 
left join Valid_pan v_pan on cln.Pan_Numbers = v_pan.Pan_Numbers





In [0]:
-- Summary reports 
with cte  as 
(Select
(Select count(*) from workspace.project.pan_number )as total_processed_records,
COUNT(*) FILTER (WHERE vw.Pan_Status = 'Valid') AS total_valid_pans,
COUNT(*) FILTER (WHERE vw.Pan_Status = 'Invalid') AS total_invalid_pans
	from  workspace.project.vw_valid_pan vw)
Select *, total_processed_records - (total_valid_pans+total_invalid_pans) as missing_incomplete_PANS from cte 

